# Demographics by Geography EDA

This notebook allows users to select multiple geographies and demographic tables and saves data in format that can be imported into Tableau Public Desktop for data visualization 

## Data Preparation
Import required libraries and make data calls that do not require user input. 


In [1]:
import pandas as pd
import os
import sqlalchemy
import datetime
import ipywidgets as widgets
from sqlalchemy.dialects.mssql.information_schema import columns

## User Input

 ### Select Year of Analysis

In [2]:
years_available = ['2022']
acs_survey_year = ''
acs_survey_year = widgets.Dropdown(
    options = years_available,
    value = '2022',
    # description = 'Select Year of Analysis',
    disabled=False
)
acs_survey_year

Dropdown(options=('2022',), value='2022')

### Select Survey
Select whether to view ACS 1yr geographies (congressional district, county, place, or PUMA), or a specific block group at the 5yr level.

In [3]:
survey_type = widgets.Dropdown(
    options=[('1-Year Estimates', 'acsse'), ('5-Year Estimate', 'acs5')],
    # description='ACS Survey Type',
    disabled=False,
    layout=widgets.Layout(width='max-content'),
    tooltips=['Congressional District, County, Place, or PUMA', 'Block Group']
)
survey_type

Dropdown(layout=Layout(width='max-content'), options=(('1-Year Estimates', 'acsse'), ('5-Year Estimate', 'acs5…

### Select Geographic Level of Analysis

In [4]:
geography_type = ''
if survey_type.value == 'acsse':
    geography_type = widgets.Dropdown(
        options = [('Congressional District', 'congressional_district'), ('County', 'county'), ('Place', 'place'), ('PUMA', 'puma')],
        value = 'county',
        # description = 'Select Geography',
        disabled=False
    )
elif survey_type.value == 'acs5':
    geography_type = ('Block Group', 'block_group')[1]
geography_type

Dropdown(index=1, options=(('Congressional District', 'congressional_district'), ('County', 'county'), ('Place…

### Select Tables From Survey for Selected Geography

In [6]:
# imports Census survey data
db_filepath = os.path.join(f'data/databases/census_{survey_type.value}_{acs_survey_year.value}_{geography_type.value}.db')

sql_engine = sqlalchemy.create_engine('sqlite:///' + db_filepath)
tables = pd.read_sql_table('tables', sql_engine)
concepts = list(tables.concept)

# adds index as second item in each tuple for loading into widget
concepts_for_widget = []
for index, concept in enumerate(concepts):
    concept = str(concept).capitalize().replace('_', ' ')
    concepts_for_widget.append((concept, index))

select_widget_tables = widgets.SelectMultiple(
    options=list(concepts_for_widget),
    rows=len(concepts_for_widget),
    # description='Select Concept',
    layout=widgets.Layout(width='max-content'),
    disabled=False
)

select_widget_tables

SelectMultiple(layout=Layout(width='max-content'), options=(('Veteran status for the civilian population 18 ye…

### Select Specific Geography

In [43]:
# loads all database data locally and conducts any further filtering from this notebook locally
selected_tables = []
for table in select_widget_tables.value:
    selected_tables.append(tables.iloc[table]['name'])

list_of_dataframes = []
if geography_type.value == 'block_group':    
    #  geography_selector = widgets.Text(
    #     value = '23893US9834092834',
    #     placeholder = '23893US9834092834',
    #     # description = 'Block Group UCGID',
    #     disabled=False
    # )
     pass
else:
    # selects name of geographies available
    for table in selected_tables:
        df = pd.read_sql_table(table, sql_engine)
        list_of_dataframes.append(df)

geography_names = list(list_of_dataframes[0]['NAME'])

select_geography_tables = widgets.SelectMultiple(
    options=geography_names,
    rows=len(geography_names),
    # description='Select Concept',
    layout=widgets.Layout(width='max-content'),
    disabled=False
)
select_geography_tables

SelectMultiple(layout=Layout(width='max-content'), options=('San Antonio City (Southwest)--Inside Loop I-410 P…

In [73]:
select_geography_tables_list = select_geography_tables.value
crosswalk_df = pd.read_sql_table('crosswalk', sql_engine)

output_path_dir = f'data/outputs/{str(datetime.datetime.now())}'
os.makedirs(output_path_dir, exist_ok=True)

for dataframe in list_of_dataframes:
    
    # subset dataframe by selected geographies in select geography list
    dataframe = dataframe[dataframe['NAME'].isin(select_geography_tables_list)]
    
    # parses table name from table column, before it gets renamed using crosswalk
    table_code = str(dataframe.columns[3]).split('_')[0]
    
    table_name_df = tables[tables['name'].isin([table_code])]
    table_name = table_name_df['concept'].iloc[0]

    # renames table column names with data from crosswalk 
    for series_name in dataframe.columns:
        if crosswalk_df['name'].str.contains(series_name).any():
            # if the crosswalk contains the name of the DataFrame column (i.e., for any data column as opposed to names, descriptors, etc. ) replace table name based on key to one based on descriptor, then strip spaces, punctuation, etc. and replace with underscores for easier data manipulation and normalization, then convert from Series object to int dtype
            new_label = str(crosswalk_df[crosswalk_df['name'].str.startswith(series_name)][['label']].values)
            new_label = new_label.replace('[', '').replace(']', '').replace('\' \'', ': ').replace('\'', '').replace('!!', ': ').replace(':', '').replace('\n', '').replace('Estimate ', '').replace('Total ', '').title()
            try:
                if series_name != 'ucgid':
                    dataframe.rename(columns={series_name: new_label}, inplace=True, errors='raise')
                    dataframe = dataframe.astype({series_name: 'int'})
            except (TypeError, ValueError, KeyError):
                pass
        elif series_name == 'NAME':
            dataframe = dataframe.rename(columns={series_name: str(series_name).title()}, errors='raise')
            
        dataframe.to_csv(os.path.join(output_path_dir, f'{table_name}.csv'))

dataframe


,index,Name,Total,Under 18 Years,18 To 24 Years,25 To 34 Years,35 To 44 Years,45 To 54 Years,55 To 64 Years,65 Years And Over,ucgid
6,18,Bexar County (South)--San Antonio City (Far So...,155224,38131,16498,25816,25524,17224,12328,19703,795P200US4805907
14,26,Bexar County (North)--San Antonio City (Far No...,159997,37102,15679,27640,23339,21142,15423,19672,795P200US4805915
